In [1]:
# Wec_grid_class 
import os, sys

paths = []
with open('../wec-grid-code/path_config.txt', 'r') as fp:
    while 1:
        line = fp.readline()
        if len(line) == 0:  # end of file break
            break
        temp = line.split('\n')
        paths.append(temp[0])

psse_path = paths[0]
wec_sim_path = paths[1]
wec_model_path = paths[2]
wec_grid_class_path = paths[3]
wec_grid_folder = paths[4]

# Path stuff

sys.path.append(psse_path + "\\PSSPY37")
sys.path.append(psse_path + "\\PSSBIN")
sys.path.append(psse_path + "\\PSSLIB")
sys.path.append(psse_path + "\\EXAMPLE")
sys.path.append(psse_path + "\\PSSPY37")
os.environ['PATH'] = (psse_path + "\\PSSPY37;" + psse_path + "\\PSSBIN;" + psse_path + "\\EXAMPLE;" + os.environ['PATH'])

import pandas as pd
import psse35
psse35.set_minor(3)
import matplotlib.pyplot as plt
import psspy
import redirect
redirect.psse2py()




class Wec_grid:
    def __init__(self, case, solver, wec_bus):
        """
        Descrtiption:
        input:
        output
        """
        psspy.psseinit(50)
        # initalized variables and files
        self.case_file = case
        self.dynamic_case_file = ""
        self.lst_param = ['BASE', 'PU', 'ANGLED', 'P', 'Q']
        self.dataframe = pd.DataFrame()
        self.wecBus_num = wec_bus
        self.history = {}
        self.solver = solver
        self._i = psspy.getdefaultint()
        self._f = psspy.getdefaultreal()
        self._s = psspy.getdefaultchar()

        # initialization functions

        if self.case_file.endswith('.sav'):
            psspy.case(self.case_file)
        elif self.case_file.endswith('.raw'):
            psspy.read(1, self.case_file)
        self.run_powerflow(self.solver)

        # program variables
        self.swingBus = self.dataframe.loc[self.dataframe['Bus'] == 'BUS 1']
        self.swingBus.insert(0, 'time', None)
        self.wecBus = self.dataframe.loc[self.dataframe['Bus'] == 'BUS {}'.format(str(self.wecBus_num))]
        self.wecBus.insert(0, 'time', None)
        self.history['Start'] = self.dataframe

    def clear(self):
        """
        Descrtiption:
        input:
        output:
        """

        # initalized variables and files
        self.lst_param = ['BASE', 'PU', 'ANGLED', 'P', 'Q']
        self.dataframe = pd.DataFrame()
        self.history = {}

        # initialization functions
        psspy.read(1, self.case_file)
        self.run_powerflow(self.solver)

        # program variables
        self.swingBus = self.dataframe.loc[self.dataframe['Bus'] == 'BUS 1']
        self.swingBus.insert(0, 'time', None)
        self.wecBus = self.dataframe.loc[self.dataframe['Bus'] == 'BUS {}'.format(str(self.wecBus_num))]
        self.wecBus.insert(0, 'time', None)
        self.history['Start'] = self.dataframe

    def run_dynamics(self):
        """
        Descrtiption:
        input:
        output:
        """

        #check if dynamic file is loaded
        if self.dynamic_case_file == "":
            self.dynamic_case_file = input("Dynamic File location")
        #self.dynamic_case_file = "../input_files/dynamics.dyr"
        # Convert Generators
        psspy.cong()
        # Solve for dynamics
        psspy.ordr()
        psspy.fact()
        psspy.tysl()
        # Add Dynamics data file
        psspy.dyre_new(dyrefile=self.dynamic_case_file)
        # Add channels and parameters
        # BUS VOLTAGE
        psspy.chsb(sid=0, all=1, status=[-1, -1, -1, 1, 13, 0])
        # Active and Reactive Power Flow
        psspy.chsb(sid=0, all=1, status=[-1, -1, -1, 1, 16, 0])

        # Save snapshoot
        path = wec_grid_folder + "\\output_files\\test.out"
        psspy.snap(sfile=path)

        # Initialize
        psspy.strt(outfile=path)

        psspy.strt()
        psspy.run(tpause=1)
        # Set simulation parameter step size
        psspy.dynamics_solution_params(realar3=0.01)

    def run_WEC_Sim(self):
        """
        Descrtiption:
        input:
        output:
        """
        # TODO add simulation time arg pass
        import matlab.engine
        eng = matlab.engine.start_matlab()
        print("Matlab Engine estbalished")
        eng.cd(wec_model_path)
        path = wec_sim_path  # Update to match your WEC-SIM source location
        eng.addpath(eng.genpath(path), nargout=0)
        print("calling W2G")
        eng.w2gSim(nargout=0)
        print("calling PSSe formatting")
        eng.WECsim_to_PSSe_dataFormatter(nargout=0)
        print("sim complete")

    def run_powerflow(self, solver):
        if solver == 'fnsl':
            psspy.fnsl()
            self.get_values()
        elif solver == 'GS':
            psspy.solv()
            self.get_values()
        elif solver == 'DC':
            psspy.dclf_2(1, 1, [1, 0, 1, 2, 1, 1], [0, 0, 0], '1')
            self.get_values()
        else:
            print("error in run_pf")

    def get_values(self):
        """
        Descrtiption: 
        input: List of Parameters such as BASE, PU, KV, ANGLED 
        output: Dataframe of the selected parameters for each bus
        """
        lst = self.lst_param
        temp_dict = {}
        for i in range(len(lst)):
            if lst[i] != "P" and lst[i] != "Q":
                ierr, bus_voltages = psspy.abusreal(-1, string=lst[i])
                bus_add = {}
                for j in range(len(bus_voltages[0])): # error here
                    bus_add['BUS {}'.format(j+1)] = bus_voltages[0][j]
                    temp_dict[lst[i]] = bus_voltages[0]
                temp_dict[lst[i]] = bus_add
                

        self.dataframe = pd.DataFrame.from_dict(temp_dict)
        self.dataframe = self.dataframe.reset_index()
        self.dataframe = self.dataframe.rename(columns={'index':"Bus"})
        self.dataframe['Type'] = psspy.abusint(-1, string="TYPE")[1][0]
        self.dataframe.insert(0, "BUS_ID", range(1, 1 + len(self.dataframe)))
        self.addGeninfo()
        self.addLoadinfo()
        
        if "P" in lst:
            self.dataframe['P'] = 0 # initalize P column
            self.get_p_or_q('P')

        if "Q" in lst:
            self.dataframe['Q'] = 0 # initalize Q column
            self.get_p_or_q('Q')
               
    def get_p_or_q(self, letter):
        """
        Descrtiption: retre P (activate) Q (reactive) Voltage (in PU) and Voltage Angle for each Bus in the current loaded case
        input:
        output:
        """
        gen = self.dataframe['{} Gen'.format(letter)]
        load = self.dataframe['{} Load'.format(letter)]
        temp = []
        for i in range(len(gen)):
            if (not pd.isnull(gen[i])) and (not pd.isnull(load[i])):
                temp.append(gen[i] - load[i])
            elif pd.isnull(gen[i]) and pd.isnull(load[i]):
                temp.append(None)
            else:
                if pd.isnull(gen[i]):
                    temp.append(-load[i])
                else:
                    temp.append(gen[i])
        self.dataframe['{}'.format(letter)] = temp
            
    def busNum(self):
        """
        Descrtiption: Returns the number of Buses in the currently loaded case
        input: None 
        output: Number of Buses
        """
        psspy.bsys(0, 0, [0.0, 0.0], 1, [1], 0, [], 0, [], 0, [])
        ierr,all_bus = psspy.abusint(0, 1, ['number'])
        return all_bus[0]
    
    def dc_injection(self, ibus, p, pf_solver, time):
        """
        Descrtiption:
        input:
        output:
        """
        ierr = psspy.machine_chng_3(ibus, "1", [], [p])
        if ierr > 0:
            print("Failed | machine_chng_3 code = {}".format(ierr))
        psspy.dclf()
        self.get_values()

        temp = pd.DataFrame(self.dataframe.loc[self.dataframe['Bus'] == 'BUS 1'])
        temp.insert(0, 'time', time)
        self.swingBus = self.swingBus.append(temp)

        temp = pd.DataFrame(self.dataframe.loc[self.dataframe['Bus'] == 'BUS {}'.format(str(self.wecBus_num))])
        temp.insert(0,'time', time)
        self.wecBus = self.wecBus.append(temp)

        self.history[time] = self.dataframe
        
    def ac_injection(self, ibus, p, v, pf_solver, time):
        """
        Descrtiption:
        input:
        output:
        """
        ierr = psspy.machine_chng_3(ibus, "1", [], [p])
        if ierr > 0:
            print("Failed | machine_chng_3 code = {}".format(ierr))
            
        ierr = psspy.bus_chng_4(ibus, 0, [],[self._f, v])
        if ierr > 0:
            print("Failed | bus_chng_4 code = {}".format(ierr))
            
        self.run_powerflow(pf_solver)
        temp = pd.DataFrame(self.dataframe.loc[self.dataframe['Bus'] == 'BUS 1'])
        temp.insert(0, 'time', time)
        self.swingBus = self.swingBus.append(temp)
        temp = pd.DataFrame(self.dataframe.loc[self.dataframe['Bus'] == 'BUS {}'.format(str(self.wecBus_num))])
        temp.insert(0,'time', time)
        self.wecBus = self.wecBus.append(temp)

        self.history[time] = self.dataframe

    def plotSwingBus(self):
        """
        Descrtiption:
        input:
        output:
        """
        # fig, axs = plt.subplots(2)
        # plt.plot(self.swingBus.time, self.swingBus[letter], marker="o", markersize=5, markerfacecolor="green")
        # plt.xlabel("Time (sec)")
        # plt.ylabel("{} in MW".format(letter))
        # plt.title("Swing bus")
        fig, (ax1, ax2) = plt.subplots(2)
        fig.suptitle("Swing bus")
        ax1.plot(self.swingBus.time, self.swingBus["P"], marker="o", markersize=5, markerfacecolor="green")
        ax2.plot(self.swingBus.time, self.swingBus["Q"], marker="o", markersize=5, markerfacecolor="green")
        ax1.set(xlabel="Time(sec)", ylabel="P(MW)")
        ax2.set(xlabel="Time(sec)", ylabel="Q(MW)")
        plt.show()
    
    def plotWecBus(self, mode="Gen"):
        """
        Descrtiption:
        input:
        output:
        """
        if mode == "Gen":
            fig, (ax1, ax2) = plt.subplots(2)
            fig.suptitle("Swing bus")
            ax1.plot(self.wecBus.time, self.wecBus["P"], marker="o", markersize=5, markerfacecolor="green")
            ax2.plot(self.wecBus.time, self.wecBus["Q"], marker="o", markersize=5, markerfacecolor="green")
            ax1.set(xlabel="Time(sec)", ylabel="P (MW)")
            ax2.set(xlabel="Time(sec)", ylabel="Q (sMW)")
            plt.show()
        else: 
            fig, (ax1, ax2) = plt.subplots(2)
            fig.suptitle("Swing bus")
            ax1.plot(self.wecBus.time, self.wecBus["P Gen"], marker="o", markersize=5, markerfacecolor="green")
            ax2.plot(self.wecBus.time, self.wecBus["Q Gen "], marker="o", markersize=5, markerfacecolor="green")
            ax1.set(xlabel="Time(sec)", ylabel="P Gen (MW)")
            ax2.set(xlabel="Time(sec)", ylabel="Q Gen (MW)")
          
    def addGeninfo(self):
        """
        Descrtiption:
        input:
        output:
        """
        buses = psspy.amachint(-1, 4,"NUMBER")[1][0]
        temp = psspy.amachcplx(-1,1,'PQGEN')
        pointer = 0 
        pointer_1 = 0
        p_gen_df_list = []
        q_gen_df_list = []

        for index, row in self.dataframe.iterrows():
            if row['BUS_ID'] == buses[pointer]:
                p_gen_df_list.append(temp[1][0][pointer].real)
                q_gen_df_list.append(temp[1][0][pointer].imag)
                pointer += 1
                if pointer >= len(buses):
                    pointer = 0
            else:
                p_gen_df_list.append(None)
                q_gen_df_list.append(None)
        self.dataframe['P Gen'] = p_gen_df_list
        self.dataframe['Q Gen'] = q_gen_df_list

    def addLoadinfo(self):
        """
        Descrtiption:
        input:
        output:
        """
        buses = psspy.aloadint(-1, 4, "NUMBER")[1][0]
        if len(buses) != 0:
            temp = psspy.aloadcplx(-1, 1, "MVAACT")
            pointer = 0
            p_load_df_list = []
            q_load_df_list = []

            for index, row in self.dataframe.iterrows():
                if row['BUS_ID'] == buses[pointer]:

                    p_load_df_list.append(temp[1][0][pointer].real)
                    q_load_df_list.append(temp[1][0][pointer].imag)
                    pointer += 1
                    if pointer >= len(buses):
                        pointer = 0
                else:
                    p_load_df_list.append(None)
                    q_load_df_list.append(None)
            self.dataframe['P Load'] = p_load_df_list
            self.dataframe['Q Load'] = q_load_df_list
        else:
            self.dataframe['P Load'] = None
            self.dataframe['Q Load'] = None

    Sets PSSE environment to latest minor version among installed versions of PSSE 35.
    Use psse35.set_minor(n) to set PSSE35 minor version (n) to use.
        Example, for PSSE 35.0.x, use this as: psse35.set_minor(0)


In [2]:
case = "../input_files/RTS96DYN/IEEE_24_bus.RAW"

In [3]:
pf = Wec_grid(case,"fnsl",[3])


 Input error detected at !
 -m ipykernel_launcher -f C:\Users\alexb\AppData\Roaming\jupyter\runtime\kernel-a70144b5-8d93-4272-9193-292b58645ec0.json
                          !

 PSS(R)E Xplore Version 35
 Copyright (c) 1976-2023
 Siemens Industry, Inc.,
 Power Technologies International                            (PTI)
 This program is a confidential  unpublished  work  created  and  first
 licensed in 1976.  It is a trade secret which is the property of  PTI.
 All use,  disclosure,  and/or reproduction not specifically authorized
 by  PTI  is prohibited.   This  program is protected  under  copyright
 laws  of  non-U.S.  countries  and  by  application  of  international
 treaties.  All  Rights  Reserved  Under  The  Copyright  Laws.


           SIEMENS POWER TECHNOLOGIES INTERNATIONAL

    50 BUS POWER SYSTEM SIMULATOR--PSS(R)E Xplore-35.3.2

             INITIATED ON MON, JAN 30 2023  20:16

 Messages for api FNSL
 No network data in memory   (005807)


TypeError: 'NoneType' object is not subscriptable

In [5]:
pf.dataframe

NameError: name 'pf' is not defined

In [65]:
psspy.read(1, case)


 Reading IC, SBASE, REV, XFRRAT, NXFRAT, BASFRQ...

 Converting RAW data from PSS(R)E 33 to PSS(R)E 34 format
 Starting processing of Case identification data records
 Finished processing of 3 records for Case identification data
 Starting processing of Bus data records
 Finished processing of 24 records for Bus data
 Starting processing of Load data records
 Finished processing of 16 records for Load data
 Starting processing of Fixed shunt data records
 Finished processing of 0 records for Fixed shunt data
 Starting processing of Generator data records
 Finished processing of 11 records for Generator data
 Starting processing of Branch data records
 Finished processing of 32 records for Branch data
 Finished processing of 0 records for System switching device data
 Starting processing of Transformer data records
 Finished processing of 24 records for 6 Transformer data
 Starting processing of Area data records
 Finished processing of 1 records for Area data
 Starting processing of T

0

In [78]:
# get_values(self)



lst = ['BASE', 'PU', 'ANGLED', 'P', 'Q']
temp_dict = {}
for bus_parameter in lst:
    if bus_parameter != "P" and bus_parameter != "Q":
        ierr, bus_parameter_values = psspy.abusreal(-1, string=bus_parameter) # grabs the bus parameter values for the specified parameter - list
        if ierr != 0:
            print("error in get_values function") 
        bus_add = {}
        for bus_index, value in enumerate(bus_parameter_values[0]): # loops over those values to create 
            bus_add['BUS {}'.format(bus_index + 1)] = value
        temp_dict[bus_parameter] = bus_add


dataframe = pd.DataFrame.from_dict(temp_dict)
dataframe = dataframe.reset_index()
dataframe = dataframe.rename(columns={'index':"Bus"})
dataframe['Type'] = psspy.abusint(-1, string="TYPE")[1][0]
dataframe.insert(0, "BUS_ID", range(1, 1 + len(dataframe)))


In [79]:
machine_bus_nums = psspy.amachint(-1, 4,"NUMBER")[1][0] #get the bus numbers of the machines - list
ierr, machine_bus_values = psspy.amachcplx(-1,1,'PQGEN') # grabs the complex values for the machine
if ierr != 0:
    raise Exception('Error in grabbing PGGEN values in addgen function')
p_gen_df_list = [None]*len(dataframe)
q_gen_df_list = [None]*len(dataframe)

for list_index, value in enumerate(machine_bus_values[0]): # iterate over the machine values 
    p_gen_df_list[machine_bus_nums[list_index] -1 ] = value.real # -1 is for the of
    q_gen_df_list[machine_bus_nums[list_index] - 1] = value.imag
    
dataframe['P Gen'] = p_gen_df_list
dataframe['Q Gen'] = q_gen_df_list

In [80]:
load_bus_nums = psspy.aloadint(-1, 4, "NUMBER")[1][0]#get the bus numbers of buses with loads - list
ierr, load_bus_values = psspy.aloadcplx(-1, 1, "MVAACT") # load values 
if ierr != 0:
    raise Exception('Error in grabbing PGGEN values in addgen function')
p_load_df_list = [None]*len(dataframe)
q_load_df_list = [None]*len(dataframe)

for list_index, value in enumerate( load_bus_values[0]): # iterate over the machine values 
    p_load_df_list[load_bus_nums[list_index] -1 ] = value.real # -1 is for the offset 
    q_load_df_list[load_bus_nums[list_index] - 1] = value.imag
dataframe['P Load'] = p_load_df_list
dataframe['Q Load'] = q_load_df_list

In [81]:
letter = 'Q'

gen_values = dataframe['{} Gen'.format(letter)] #
load_values = dataframe['{} Load'.format(letter)]
letter_list = [None] * len(dataframe)

for i in range(len(letter_list)):
    gen = gen_values[i]
    load = load_values[i]
    
    if ( not pd.isnull(gen) ) and ( not pd.isnull(load) ):
        letter_list[i] = gen - load
    else:
        if (not pd.isnull(gen)):
            letter_list[i] = gen
        if (not pd.isnull(load)):
            letter_list[i] = 0 - load # gen is 
    

dataframe['{}'.format(letter)] = letter_list

letter = 'P'

gen_values = dataframe['{} Gen'.format(letter)] #
load_values = dataframe['{} Load'.format(letter)]
letter_list = [None] * len(dataframe)

for i in range(len(letter_list)):
    gen = gen_values[i]
    load = load_values[i]
    
    if ( not pd.isnull(gen) ) and ( not pd.isnull(load) ):
        letter_list[i] = gen - load
    else:
        if (not pd.isnull(gen)):
            letter_list[i] = gen
        if (not pd.isnull(load)):
            letter_list[i] = 0 - load # gen is 
    

dataframe['{}'.format(letter)] = letter_list

In [70]:
old = dataframe

In [71]:
old.iloc[[0,1,12]]

,BUS_ID,Bus,BASE,PU,ANGLED,Type,P Gen,Q Gen,P Load,Q Load,Q,P
0,1,BUS 1,138.0,1.00000,0.0000,3,35.848999,180.356995,NaN,NaN,180.356995,35.848999
1,2,BUS 2,138.0,0.99783,0.0102,2,67.000000,0.000000,97.0,20.000000,-20.000000,-30.000000
12,13,BUS 13,230.0,0.83731,15.4352,2,200.000000,0.000000,130.0,27.000002,-27.000002,70.000000


In [82]:
dataframe.iloc[[0,1,12]]

,BUS_ID,Bus,BASE,PU,ANGLED,Type,P Gen,Q Gen,P Load,Q Load,Q,P
0,1,BUS 1,138.0,1.000000,0.000000,3,256.340637,121.621681,NaN,NaN,121.621681,256.340637
1,2,BUS 2,138.0,0.997222,-0.090895,2,20.284201,0.000000,97.0,20.000000,-20.000000,-76.715799
12,13,BUS 13,230.0,0.838016,2.771898,2,20.284201,0.000000,130.0,27.000002,-27.000002,-109.715799


In [73]:
import sqlite3

In [74]:
con = sqlite3.connect("../input_files/r2g_database.db")
injection = pd.read_sql_query("SELECT * from WEC_output", con)
print(injection)

      time  ibus         pg   vs         pt   pb         qt         qb
0      0.0   1.0   0.000000  1.0   0.000000  0.0   0.000000   0.000000
1    450.0   1.0  20.284200  1.1  20.284200  0.0  22.064601 -22.064601
2    750.0   1.0  21.086833  1.1  21.086833  0.0  21.173366 -21.173366
3   1050.0   1.0  17.886427  1.1  17.886427  0.0  23.649517 -23.649517
4   1350.0   1.0  14.244265  1.1  14.244265  0.0  26.306681 -26.306681
5   1650.0   1.0  21.451810  1.1  21.451810  0.0  20.855884 -20.855884
6   1950.0   1.0  19.953674  1.1  19.953674  0.0  22.165508 -22.165508
7   2250.0   1.0  17.727287  1.1  17.727287  0.0  24.159891 -24.159891
8   2550.0   1.0  13.796929  1.1  13.796929  0.0  26.609123 -26.609123
9   2850.0   1.0  13.583606  1.1  13.583606  0.0  26.734918 -26.734918
10  3150.0   1.0  15.860482  1.1  15.860482  0.0  25.432033 -25.432033
11  3450.0   1.0  17.103224  1.1  17.103224  0.0  24.629206 -24.629206


In [75]:
ibuses = [2,13]


In [76]:
p = [20.284199882370412, 20.284199882370412]
v = [1.1, 1.1]

In [77]:
for idx, bus in enumerate(ibuses):
    ierr = psspy.machine_data_2(bus, '1', realar1=p[idx])
    if ierr > 0:
        print("Failed | machine_chng_3 code = {}".format(ierr))

    ierr =  psspy.bus_chng_4(bus, 0, realar2=v[idx])
    if ierr > 0:
        print("Failed | bus_chng_4 code = {}".format(ierr))
psspy.fnsl()


 Power flow data changed for machine "1" at bus 2 [2           138.00]:
 X--ORIGINAL--X  X-NEW VALUE--X  DATA ITEM
    67.0000         20.2842      PG

 Power flow data changed for bus 2 [2           138.00]:
 X--ORIGINAL--X  X-NEW VALUE--X  DATA ITEM
   0.997830         1.10000      VM

 Power flow data changed for machine "1" at bus 13 [13          230.00]:
 X--ORIGINAL--X  X-NEW VALUE--X  DATA ITEM
    200.000         20.2842      PG

 Power flow data changed for bus 13 [13          230.00]:
 X--ORIGINAL--X  X-NEW VALUE--X  DATA ITEM
   0.837310         1.10000      VM

 Ordering network...
 Diagonals = 23   Off-diagonals = 45   Maximum size = 62

  ITER       DELTAP      BUS         DELTAQ      BUS        DELTA/V/      BUS       DELTAANG      BUS
   0        40.0774(      2     )   70.3675(      2     )
                                                             0.86146(     12     )   0.80574(     21     )
   1         5.2805(     23     )    7.9884(     12     )
               

0